<a id="toc"></a>
# Table of Contents
## [Python for Data Analysis (pandas)](#pandas)
## [Querying Dataframes](#querying)
### [Query Challenge](#query_challenge)
## [Reshaping Data](#reshaping)
## [Aggregating Data](#aggregates)
## [Merging Datasets](#merging)
## [Ordering Data](#ordering)
## [Exporting Data](#export)

<a id="pandas"></a>
## Pandas
[Back to Table of Contents](#toc)

Pandas is a _library_ which allows us to do some powerful operations with table-like data. We can query datasets with a high degree of granularity, merge them together, sort, and aggregate them.

I highly suggest you take a look and read through the documentation on it when you can! **[It is available here.](https://pandas.pydata.org/pandas-docs/stable/index.html)**

Combined with this lesson are 4 Excel spreadsheets that we will be combining together to form various queries.

- `COMPANIES.xlsx`
- `EMPLOYEES.xlsx`
- `FAKE_DATA_BUILD.xlsx` <- This is our main table
- `ISO_COUNTRY_LOOKUPS.xlsx`

To start with, let's read a spreadsheet and see the output.

Remember, if we want to use a _library_ we have to **import** it.

In [1]:
import pandas as pd # we can create a temporary name using the "as" keyword here which can make it shorter

pd.set_option('display.max_rows', 1000) # We can use this variable to decide how many rows we want to see
pd.set_option('display.max_columns', 500) # We can use this variable to decide how many columns we want to see

df_employees = pd.read_excel('EMPLOYEES.xlsx')
df_employees # This will display the table

,EMPLOYEE_ID,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,QZE4337,Domingos,Bello,1964-09-13,BL
1,QVJ3467,Kira,Gomes,1990-06-12,MF
2,NCD9017,Modesto,Fernández,1974-06-28,MQ
3,DHI8102,Samuel,Escárcega,1996-09-08,MQ
4,VDS7914,Raya,Bustillo,1983-11-30,MQ
5,EWJ3178,Lavrentiy,Gallo,1982-05-13,GP
6,KFW4978,Ofelia,Rojas,1995-08-10,GP
7,ZJQ3093,Estela,Amador,1979-07-17,GP
8,YVK2375,Leonti,Fernández,1996-11-22,GP
9,OGQ7027,Lourenço,Fernández,1971-01-18,GP


Above, I've named the table "df_employees". Since this table is going to hold the employee information, I've called it "employees" but I've added the prefix "df_", why?

In pandas, table objects are stored as what are called "dataframes" - I'm using "df" as shorthand for that to let me know what kind of _object_ I'm working with.

Let's read our other sheets into dataframe objects so we can begin using them.

In [2]:
df_main = pd.read_excel("FAKE_DATA_BUILD.xlsx", sheet_name="Main")
df_main_definitions = pd.read_excel("FAKE_DATA_BUILD.xlsx", sheet_name="VariableDefinitions")
df_companies = pd.read_excel("COMPANIES.xlsx")
df_countries = pd.read_excel("ISO_COUNTRY_LOOKUPS.xlsx")

In [3]:
# write each dataframe name here individually and see what it outputs when you run this

df_main_definitions

,DATE,Date that the transaction originated
0,ORGN_CTRY_CODE,The ISO 3166-2 code of the exporting country
1,DEST_CTRY_CODE,The ISO 3166-2 code of the importing country
2,PKG_HT,Height of the package
3,PKG_LT,Length of the package
4,PKG_WDT,Width of the package
5,TRANSPO_TYPE,Type of transportation
6,CUSTOMS_TYPE,Type of customs declaration
7,DECLARED_VALUE_USD,Declared value of the shipment
8,SOLUTION_TYPE,Type of solution (FF or SP)
9,SALES_CTRY_CODE,The ISO 3166-2 code of the owner sales team


Next, we can learn a little bit about our data using a few small tools:

- `len()` <- tells us how many rows are in the dataframe.
- `dataframe.info()` <- gives us basic information about the dataframe.
- `dataframe.columns.values` <- Gives us the headers of the dataframe (useful later).

In [4]:
print('Number of rows in df_main: ' + str(len(df_main)))
print('Number of rows in df_employees: ' + str(len(df_employees)))
print('Number of rows in df_companies: ' + str(len(df_companies)))
print('Number of rows in df_countries: ' + str(len(df_countries)))

Number of rows in df_main: 44232
Number of rows in df_employees: 486
Number of rows in df_companies: 1950
Number of rows in df_countries: 249


Let's take a look at df_main and see what we can learn about our data.

In [5]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44232 entries, 0 to 44231
Data columns (total 17 columns):
DATETIME              44232 non-null datetime64[ns]
ORGN_CTRY_CODE        44232 non-null object
DEST_CTRY_CODE        44232 non-null object
PKG_LENGTH            44232 non-null int64
PKG_HEIGHT            44232 non-null int64
PKG_WIDTH             44232 non-null int64
TRANSPO_TYPE          44232 non-null object
CUSTOMS_TYPE          44232 non-null object
DECLARED_VALUE_USD    44232 non-null int64
SOLUTION_TYPE         44232 non-null object
SALES_CTRY_CODE       44232 non-null object
EMPLOYEE_ID           44232 non-null object
COMPANY_ID            44232 non-null object
INVOICE_NUM           44232 non-null object
INVOICE_REV           44232 non-null float64
INVOICE_PAID          44232 non-null bool
DATE_PAID             35451 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(1), int64(4), object(9)
memory usage: 5.4+ MB


In [6]:
df_main.columns.values

array(['DATETIME', 'ORGN_CTRY_CODE', 'DEST_CTRY_CODE', 'PKG_LENGTH',
       'PKG_HEIGHT', 'PKG_WIDTH', 'TRANSPO_TYPE', 'CUSTOMS_TYPE',
       'DECLARED_VALUE_USD', 'SOLUTION_TYPE', 'SALES_CTRY_CODE',
       'EMPLOYEE_ID', 'COMPANY_ID', 'INVOICE_NUM', 'INVOICE_REV',
       'INVOICE_PAID', 'DATE_PAID'], dtype=object)

Although looking at the data is useful, we need to learn a little more about what each column contains. In this dataset, we have a "VariableDefinitions" sheet what we pulled into df_main_definitions. Let's take a look.

In [7]:
df_main_definitions

,DATE,Date that the transaction originated
0,ORGN_CTRY_CODE,The ISO 3166-2 code of the exporting country
1,DEST_CTRY_CODE,The ISO 3166-2 code of the importing country
2,PKG_HT,Height of the package
3,PKG_LT,Length of the package
4,PKG_WDT,Width of the package
5,TRANSPO_TYPE,Type of transportation
6,CUSTOMS_TYPE,Type of customs declaration
7,DECLARED_VALUE_USD,Declared value of the shipment
8,SOLUTION_TYPE,Type of solution (FF or SP)
9,SALES_CTRY_CODE,The ISO 3166-2 code of the owner sales team


<a id="querying"></a>
## Querying Dataframes
[Back to Table of Contents](#toc)

Reading data is useful but without the ability to ask a dataset questions, it doesn't really give us much more over Excel. This is where the strength of pandas starts to show.

The _syntax_ for querying is fairly simple but can get complex based upon the query itself.

For our purposes, I'll describe it this way:

The statement: `df_main[df_main["ORGN_CTRY_CODE"] == "BR"]` can be read as "within the dataframe df_main tell me where 'ORGN_CTRY_CODE' equals 'BR'." Or, more simply: all data for exports from Brazile.

We can also utilize multiple conditions like so:

`df_main[(df_main["ORGN_CTRY_CODE"] == "BR") & (df_main["DEST_CTRY_CODE"] == "US")]` - this can be read as "within the dataframe df_main tell me where 'ORGN_CTRY_CODE' equals 'BR' **_and_** 'DEST_CTRY_CODE' equals 'US'. Or, more simply, all data for packages exported from Brazil to the US.


In [ ]:
df_main[df_main["ORGN_CTRY_CODE"] == "BR"]

In [ ]:
df_main[(df_main["ORGN_CTRY_CODE"] == "BR") & (df_main["DEST_CTRY_CODE"] == "US")]

<a id="query_challenge"></a>
### Query Challenge
[Back to Table of Contents](#toc)

Write the following queries:

- Get information about packages _from_ Mexico that were of transportation type "Truck"
- Get information about packages from Argentina whose invoices have _NOT_ been paid.
- Get information about packages from the US for 2017
    - Query dates like this: `(df_main["DATETIME"] >= '01-01-2017') & (df_main["DATETIME"] < '01-01-2018')`
- Get information about packages for all countries whose "SOLUTION_TYPE" was "FF" that have "INVOICE_REV" of greater than 500

In [8]:
df_main[(df_main["ORGN_CTRY_CODE"] == "MX") & (df_main["TRANSPO_TYPE"] == "Truck")]

,DATETIME,ORGN_CTRY_CODE,DEST_CTRY_CODE,PKG_LENGTH,PKG_HEIGHT,PKG_WIDTH,TRANSPO_TYPE,CUSTOMS_TYPE,DECLARED_VALUE_USD,SOLUTION_TYPE,SALES_CTRY_CODE,EMPLOYEE_ID,COMPANY_ID,INVOICE_NUM,INVOICE_REV,INVOICE_PAID,DATE_PAID
9,2016-03-28 07:03:09.061,MX,US,60,5,13,Truck,INFORMAL,1715,SP,US,CNM2808,148976-GTZ,T424572808018583152,24.60,False,NaT
65,2013-02-04 13:16:47.418,MX,GT,48,55,40,Truck,FORMAL,4871,FF,MX,GVG1704,718723-YUV,T413101704027788180,560.55,True,2013-06-06 13:16:47.418
320,2013-12-10 19:59:22.619,MX,PE,33,12,25,Truck,INFORMAL,232,SP,PE,FEZ5359,964722-RAD,T416195359018069135,20.67,True,2014-02-14 19:59:22.619
333,2019-06-14 03:47:33.856,MX,US,48,57,40,Truck,INFORMAL,2636,FF,MX,UAK6034,941675-NMX,T436306034027788146,421.80,True,2019-06-28 03:47:33.856
400,2014-03-09 03:48:05.576,MX,SV,108,14,12,Truck,FORMAL,5903,SP,MX,HKU4912,708176-SNE,T417074912017788181,160.95,True,2014-09-05 03:48:05.576
470,2016-06-25 10:27:04.520,MX,DO,100,2,11,Truck,INFORMAL,2511,SP,MX,ULJ7903,638607-KQC,T425467903017788142,22.20,True,2016-06-30 10:27:04.520
554,2013-09-20 19:55:38.409,MX,PY,105,13,15,Truck,INFORMAL,2432,SP,PY,QSA2978,470838-HQT,T415382978018089177,20.38,True,2013-10-04 19:55:38.409
639,2017-04-08 11:54:18.358,MX,CA,37,23,10,Truck,FORMAL,5971,SP,MX,UAK6034,826793-EVC,T428336034017788100,160.95,True,2017-09-11 11:54:18.358
643,2017-07-01 12:59:25.360,MX,BR,90,8,2,Truck,INFORMAL,1017,SP,MX,ZMK2036,534229-QVV,T429182036017788115,22.20,True,2017-07-22 12:59:25.360
667,2018-04-13 03:50:00.903,MX,US,55,25,14,Truck,INFORMAL,2926,SP,MX,CJB8655,826793-EVC,T432038655017788175,22.20,True,2018-09-05 03:50:00.903


<a id="reshaping"></a>
## Reshaping Data
[Back to Table of Contents](#toc)

Remember viewing the output of `dataframe.columns.values`? Let's use that to slim down the actual data we want for display.

We can get a subset of columns of a dataframe like this:

- `df_new` has headers `["col1", "col2", "col3", "col4"]`.
- We get a subset of these columns by creating a list of the columns we want and feeding it to the dataframe.
    - `df_new_subset = df_new[["col2", "col4"]]`

Let's get the headers of `df_main` again and take a look to see which ones we want - we'll convert them to a list as well.

In [9]:
hdrs_main = list(df_main.columns.values)
hdrs_main

['DATETIME',
 'ORGN_CTRY_CODE',
 'DEST_CTRY_CODE',
 'PKG_LENGTH',
 'PKG_HEIGHT',
 'PKG_WIDTH',
 'TRANSPO_TYPE',
 'CUSTOMS_TYPE',
 'DECLARED_VALUE_USD',
 'SOLUTION_TYPE',
 'SALES_CTRY_CODE',
 'EMPLOYEE_ID',
 'COMPANY_ID',
 'INVOICE_NUM',
 'INVOICE_REV',
 'INVOICE_PAID',
 'DATE_PAID']

Let's say our end goal is to get the total revenue for each employee. What columns would we need to do that? We'd probably need the employee ID and the invoice revenue - two columns "EMPLOYEE_ID" and "INVOICE_REV".

In [10]:
df_employee_revenue = df_main[["EMPLOYEE_ID", "INVOICE_REV"]]
df_employee_revenue

,EMPLOYEE_ID,INVOICE_REV
0,FOG3270,24.60
1,VWH4794,147.08
2,CDV8654,146.39
3,ONG7753,20.67
4,OTZ5266,20.45
...,...,...
44227,OGQ7027,24.60
44228,PKF5901,20.57
44229,JEI4885,20.51
44230,IHE8631,150.09


<a id="aggregates"></a>
## Aggregating Data
[Back to Table of Contents](#toc)

It's great that we can get the columns that we need but there are two problems.
1. This is each individual transaction, it doesn't help us see the _total_ revenue for each employee.
2. We don't know who any of these employee IDs refer to (more on this later).

First, we need to _aggregate_ the data by summing the revenue for each employee.

The syntax is as follows:

`df_employee_revenue.groupby(["EMPLOYEE_ID"])["INVOICE_REV"].sum()`

We can also get the _average_ by using a different function:

`df_employee_revenue.groupby(["EMPLOYEE_ID"])["INVOICE_REV"].mean()`

Let's try it out.

In [11]:
df_tot_emp_rev = df_employee_revenue.groupby(["EMPLOYEE_ID"], as_index=False)["INVOICE_REV"].sum()
df_tot_emp_rev

,EMPLOYEE_ID,INVOICE_REV
0,AFG2372,6769.81
1,AIU2281,9696.23
2,AKN2433,7802.30
3,AKU3205,9874.56
4,ANI6151,8690.72
5,ANP8287,7163.46
6,AQN8786,9616.06
7,ARW9403,8990.35
8,ASH6956,10535.80
9,AYJ4798,9796.45


<a id="merging"></a>
## Merging Datasets
[Back to Table of Contents](#toc)

Aggregating data is useful but it doesn't solve the 2nd problem we mentioned earlier: we don't know who these people are. Luckily, we have another spreadsheet which has all of the employee information we need to solve this problem.

The way that we handle situations like this is to use pandas' _merge_ functionality.

Here's the syntax:

`df1.merge(df2, left_on='lkey', right_on='rkey')`

- **df1** is the dataset you want to merge outside data onto.
- **df2** is the dataset you want to merge onto df1.
- **left_on** is the name of the column in df1 you want df2 to merge onto.
- **right_on** is the name of the column in df2 you want to connect to the column in df1.
- If you don't put anything for the keys, it will attempt to find a matching pair of keys.

[Additional information can be found here.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Let's merge df_employees onto df_employee_revenue.

In [12]:
# Let's take a look at df_employees again to verify the columns and data
df_employees

,EMPLOYEE_ID,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,QZE4337,Domingos,Bello,1964-09-13,BL
1,QVJ3467,Kira,Gomes,1990-06-12,MF
2,NCD9017,Modesto,Fernández,1974-06-28,MQ
3,DHI8102,Samuel,Escárcega,1996-09-08,MQ
4,VDS7914,Raya,Bustillo,1983-11-30,MQ
5,EWJ3178,Lavrentiy,Gallo,1982-05-13,GP
6,KFW4978,Ofelia,Rojas,1995-08-10,GP
7,ZJQ3093,Estela,Amador,1979-07-17,GP
8,YVK2375,Leonti,Fernández,1996-11-22,GP
9,OGQ7027,Lourenço,Fernández,1971-01-18,GP


In [13]:
df_tot_emp_rev

,EMPLOYEE_ID,INVOICE_REV
0,AFG2372,6769.81
1,AIU2281,9696.23
2,AKN2433,7802.30
3,AKU3205,9874.56
4,ANI6151,8690.72
5,ANP8287,7163.46
6,AQN8786,9616.06
7,ARW9403,8990.35
8,ASH6956,10535.80
9,AYJ4798,9796.45


In [14]:
# Let's merge
df_employee_rev_info = df_tot_emp_rev.merge(df_employees, left_on="EMPLOYEE_ID", right_on="EMPLOYEE_ID")

df_employee_rev_info

,EMPLOYEE_ID,INVOICE_REV,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,AFG2372,6769.81,Lalo,Shwetz,1990-06-17,MX
1,AIU2281,9696.23,Quim,De León,1995-04-16,MX
2,AKN2433,7802.30,Gustavo,Rocha,1994-10-20,AR
3,AKU3205,9874.56,Benigno,Del Olmo,1981-08-06,AR
4,ANI6151,8690.72,Alyona,Herrero,1993-07-07,PY
5,ANP8287,7163.46,Timur,Fernández,1992-11-19,BR
6,AQN8786,9616.06,Lavrentiy,Araujo,1977-04-11,CA
7,ARW9403,8990.35,Estela,Petrov,1992-12-27,US
8,ASH6956,10535.80,Ofelia,Amador,1977-02-06,CA
9,AYJ4798,9796.45,Hermínio,Antúnez,1971-12-21,BR


<a id="ordering"></a>
## Ordering Data
[Back to Table of Contents](#toc)

Now that we've aggregated and merged out data, let's order our data and find the top 10 employees in our company! [More information is available here.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

The syntax for sorting values is as follows:

`df.sort_values(by=['col1'])`

The column we want to sort by is "INVOICE_REV".

In [ ]:
df_employee_rev_info = df_employee_rev_info.sort_values(by=["INVOICE_REV"], ascending=False)
df_employee_rev_info

Let's get the top 10.

In [ ]:
df_top_10 = df_employee_rev_info.head(10)
df_top_10

<a id="exporting"></a>
## Exporting Data
[Back to Table of Contents](#toc)

Now that we've created our report, let's export it into Excel!

In [ ]:
df_top_10.to_excel("our_cool_employee_report.xlsx", index=False)